In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
from datetime import date, timedelta
import pandas as pd
import numpy as np

def obtener_datos_yf(tickers, start_date, end_date):
    """
    Downloads historical price data for the specified tickers and date range using yfinance.

    Args:
        tickers (list): A list of ticker symbols.
        start_date (datetime.date): The start date for data download.
        end_date (datetime.date): The end date for data download.

    Returns:
        dict: A dictionary where keys are ticker symbols and values are pandas DataFrames
              containing the historical data.
    """
    print("Loading data...")
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start_date, end=end_date)
            if df.empty:
                print(f"Warning: No data downloaded for ticker: {ticker}. Skipping.")
                continue
            if 'Close' not in df.columns:
                print(f"Warning: 'Close' column not found for ticker: {ticker}. Skipping.")
                continue
            data[ticker] = df
        except Exception as e:
            print(f"An error occurred while downloading data for {ticker}: {e}")
    print("Data loading complete.")
    print("Loaded data keys:", data.keys())
    return data

def calcular_volatilidad(data):
    """
    Calculates various volatility metrics for the loaded data.

    Args:
        data (dict): A dictionary where keys are ticker symbols and values are pandas DataFrames
                     containing historical data with 'Close', 'High', and 'Low' columns.

    Returns:
        pandas.DataFrame: A DataFrame where the index is ticker symbols and columns are
                          volatility metrics ('daily_volatility', 'annualized_volatility', 'ATR', 'EWMA_volatility').
    """
    print("\nCalculating volatility data...")
    volatility_data = {}
    for ticker, df in data.items():
        if 'Close' not in df.columns or 'High' not in df.columns or 'Low' not in df.columns:
            print(f"Skipping volatility calculation for {ticker} due to missing columns.")
            continue

        df['daily_change'] = df['Close'].pct_change()
        daily_vol = df['daily_change'].std()
        annualized_vol = daily_vol * np.sqrt(252)
        df['TR'] = np.maximum(df['High'] - df['Low'], np.abs(df['High'] - df['Close'].shift(1)), np.abs(df['Low'] - df['Close'].shift(1))).fillna(0)
        atr_period = 14
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
        span = 20
        ewma_vol = df['daily_change'].dropna().ewm(span=span).std().iloc[-1] if not df['daily_change'].dropna().empty else np.nan

        volatility_data[ticker] = {
            'daily_volatility': daily_vol,
            'annualized_volatility': annualized_vol,
            'ATR': df['ATR'].iloc[-1] if not df['ATR'].dropna().empty else np.nan,
            'EWMA_volatility': ewma_vol
        }

    volatility_df = pd.DataFrame.from_dict(volatility_data, orient='index')
    print("Volatility calculation complete.")
    print("\nCalculated volatility data:")
    display(volatility_df)
    return volatility_df

def graficar_radar(volatility_df, tickers_to_plot=None, metrics_to_plot=None):
    """
    Creates a Plotly radar chart for selected tickers and volatility metrics.

    Args:
        volatility_df (pandas.DataFrame): DataFrame containing volatility metrics.
        tickers_to_plot (list, optional): A list of ticker names to include. Defaults to all loaded tickers.
        metrics_to_plot (list, optional): A list of volatility metric names to include. Defaults to all calculated metrics.
    """
    if volatility_df.empty:
        print("No volatility data available to plot.")
        return

    if tickers_to_plot is None:
        tickers_to_plot = volatility_df.index.tolist()

    if metrics_to_plot is None:
        metrics_to_plot = volatility_df.columns.tolist()

    filtered_volatility_df = volatility_df.loc[tickers_to_plot, metrics_to_plot].copy()
    filtered_volatility_df.fillna(0, inplace=True)

    if filtered_volatility_df.empty:
        print("No data available for the selected tickers and metrics after filtering.")
        return

    metrics = filtered_volatility_df.columns.tolist()
    tickers = filtered_volatility_df.index.tolist()
    max_volatility = filtered_volatility_df.replace([np.inf, -np.inf], np.nan).values.max() if not filtered_volatility_df.empty else 1
    if pd.isna(max_volatility):
        max_volatility = 1

    fig = go.Figure()

    for ticker in tickers:
        fig.add_trace(go.Scatterpolar(
            r=filtered_volatility_df.loc[ticker, metrics].values,
            theta=metrics,
            fill='toself',
            name=ticker
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, max_volatility * 1.1]
            )),
        showlegend=True,
        title="Volatility Radar"
    )
    fig.show()

def graficar_barras(volatility_df, tickers_to_plot=None):
    """
    Generates a horizontal bar chart comparing annualized volatility for selected tickers.

    Args:
        volatility_df (pandas.DataFrame): DataFrame containing volatility metrics.
        tickers_to_plot (list, optional): A list of ticker names to include. Defaults to all loaded tickers.
    """
    if volatility_df.empty:
        print("No volatility data available to plot.")
        return

    if tickers_to_plot is None:
         tickers_to_plot = volatility_df.index.tolist()

    if 'annualized_volatility' not in volatility_df.columns:
        print("Annualized volatility data not available.")
        return

    plot_data = volatility_df.loc[tickers_to_plot, 'annualized_volatility'].dropna().sort_values(ascending=False)

    if plot_data.empty:
        print("No valid annualized volatility data to plot for the selected tickers.")
        return

    plt.figure(figsize=(10, max(2, len(plot_data) * 0.5)))
    sns.barplot(x=plot_data.values, y=plot_data.index, palette='viridis')
    plt.xlabel('Annualized Volatility')
    plt.ylabel('Ticker')
    plt.title('Annualized Volatility Comparison')
    plt.tight_layout()
    plt.show()

def graficar_heatmap(volatility_df, tickers_to_plot=None, metrics_to_plot=None):
    """
    Generates a heatmap of volatility metrics for selected tickers and metrics.

    Args:
        volatility_df (pandas.DataFrame): DataFrame containing volatility metrics.
        tickers_to_plot (list, optional): A list of ticker names to include. Defaults to all loaded tickers.
        metrics_to_plot (list, optional): A list of volatility metric names to include. Defaults to all calculated metrics.
    """
    if volatility_df.empty:
        print("No volatility data available to plot.")
        return

    if tickers_to_plot is None:
        tickers_to_plot = volatility_df.index.tolist()

    if metrics_to_plot is None:
        metrics_to_plot = volatility_df.columns.tolist()

    heatmap_data = volatility_df.loc[tickers_to_plot, metrics_to_plot].copy()
    heatmap_data = heatmap_data.dropna()

    if heatmap_data.empty:
        print("No complete volatility data available for heatmap after filtering and dropping NaNs.")
        return

    plt.figure(figsize=(max(8, len(metrics_to_plot) * 2), max(5, len(heatmap_data) * 0.8)))
    sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt=".2f")
    plt.title('Volatility Metrics Heatmap')
    plt.xlabel('Volatility Metric')
    plt.ylabel('Ticker')
    plt.tight_layout()
    plt.show()

def exportar_resultados_csv(volatility_df, filename='volatility_results.csv'):
    """
    Exports the volatility results DataFrame to a CSV file.

    Args:
        volatility_df (pandas.DataFrame): DataFrame containing volatility metrics.
        filename (str, optional): The name of the CSV file. Defaults to 'volatility_results.csv'.
    """
    if volatility_df.empty:
        print("No volatility data to export.")
        return
    volatility_df.to_csv(filename)
    print(f"Volatility results exported to {filename}")

def graficar_precio_volatilidad(data, ticker, rolling_window=20, volatility_threshold_multiplier=1.5):
    """
    Plots the price history of a selected ticker and highlights periods of high volatility.

    Args:
        data (dict): A dictionary where keys are ticker symbols and values are pandas DataFrames
                     containing historical data with 'Close' columns.
        ticker (str): The ticker symbol to plot.
        rolling_window (int, optional): The window size for rolling volatility calculation. Defaults to 20.
        volatility_threshold_multiplier (float, optional): Multiplier for the average rolling
                                                          volatility to determine the high volatility threshold. Defaults to 1.5.
    """
    if ticker not in data:
        print(f"Data not found for ticker: {ticker}")
        return

    df = data[ticker].copy()

    if df.empty or 'Close' not in df.columns:
        print(f"Insufficient data for ticker: {ticker} to plot price and volatility.")
        return

    # Calculate daily returns and rolling volatility
    df['daily_returns'] = df['Close'].pct_change()
    df['rolling_volatility'] = df['daily_returns'].rolling(window=rolling_window).std().dropna()

    if df['rolling_volatility'].empty:
         print(f"Rolling volatility could not be calculated for ticker: {ticker} with window {rolling_window}.")
         return

    # Define high volatility threshold
    volatility_threshold = df['rolling_volatility'].mean() * volatility_threshold_multiplier
    df['high_volatility'] = df['rolling_volatility'] > volatility_threshold

    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=(f'{ticker} Price History', f'{ticker} Rolling Volatility ({rolling_window}-day)'))

    # Add price trace with color-coded high volatility periods
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Close'],
        mode='lines',
        name='Close Price',
        line=dict(color='blue')
    ), row=1, col=1)

    # Add markers for high volatility periods
    high_vol_dates = df[df['high_volatility']].index
    high_vol_prices = df[df['high_volatility']]['Close']

    fig.add_trace(go.Scatter(
        x=high_vol_dates,
        y=high_vol_prices,
        mode='markers',
        marker=dict(color='red', size=8, symbol='circle', opacity=0.6),
        name=f'High Volatility (>{volatility_threshold:.4f})'
    ), row=1, col=1)


    # Add rolling volatility trace
    fig.add_trace(go.Scatter(
        x=df['rolling_volatility'].index,
        y=df['rolling_volatility'],
        mode='lines',
        name='Rolling Volatility',
        line=dict(color='orange')
    ), row=2, col=1)

    # Add threshold line to volatility plot
    fig.add_shape(type="line",
        x0=df.index.min(), y0=volatility_threshold, x1=df.index.max(), y1=volatility_threshold,
        line=dict(color="red", dash="dash"),
        xref='x2', yref='y2'
    )

    fig.update_layout(height=600, title_text=f"Price and Volatility for {ticker}")
    fig.show()


class VolatilityRadar:
    def __init__(self, tickers, start_date=None, end_date=None):
        self.tickers = tickers
        self.start_date = start_date if start_date else date.today() - timedelta(days=90)
        self.end_date = end_date if end_date else date.today()
        self.data = {}
        self.volatility_df = pd.DataFrame()
        self.ticker_select = None
        self.metric_select = None
        self.graph_type_dropdown = None
        self.generate_button = None
        self.export_button = None


    def load_and_calculate(self):
        """Loads data and calculates volatility metrics."""
        self.data = obtener_datos_yf(self.tickers, self.start_date, self.end_date)
        self.volatility_df = calcular_volatilidad(self.data)
        # Update widget options after data is loaded
        if self.ticker_select:
             self.ticker_select.options = self.volatility_df.index.tolist()
        if self.metric_select:
             self.metric_select.options = self.volatility_df.columns.tolist()


    def create_widgets(self):
        """Creates and displays ipywidgets for user interaction."""
        if self.volatility_df.empty:
            print("Volatility data not available. Please load data and calculate metrics first.")
            return

        self.ticker_select = widgets.SelectMultiple(
            options=self.volatility_df.index.tolist(),
            description='Select Tickers:',
            disabled=False
        )

        self.metric_select = widgets.SelectMultiple(
            options=self.volatility_df.columns.tolist(),
            description='Select Metrics:',
            disabled=False
        )

        self.graph_type_dropdown = widgets.Dropdown(
            options=['Radar', 'Bar Chart', 'Heatmap', 'Price with Volatility'],
            description='Select Graph Type:',
            disabled=False
        )

        self.generate_button = widgets.Button(
            description='Generate Plot',
            disabled=False,
            button_style='',
            tooltip='Click to generate the selected plot',
            icon='chart-bar'
        )
        self.generate_button.on_click(self.on_button_click)

        self.export_button = widgets.Button(
            description='Export to CSV',
            disabled=False,
            button_style='success',
            tooltip='Click to export volatility results to CSV',
            icon='download'
        )
        self.export_button.on_click(self.on_export_button_click)


        # Display the widgets
        display(widgets.VBox([
            self.ticker_select,
            self.metric_select,
            self.graph_type_dropdown,
            widgets.HBox([self.generate_button, self.export_button])
        ]))

    def on_button_click(self, b):
        """Handles the button click event and triggers plotting based on selections."""
        selected_tickers = list(self.ticker_select.value) # Convert tuple to list
        selected_metrics = list(self.metric_select.value) # Convert tuple to list
        selected_graph_type = self.graph_type_dropdown.value

        if not selected_tickers:
            print("Please select at least one ticker.")
            return

        if selected_graph_type != 'Bar Chart' and selected_graph_type != 'Price with Volatility' and not selected_metrics:
             print("Please select at least one metric for Radar or Heatmap.")
             return

        print(f"\nGenerating {selected_graph_type}...")

        if selected_graph_type == 'Radar':
            graficar_radar(self.volatility_df, tickers_to_plot=selected_tickers, metrics_to_plot=selected_metrics)
        elif selected_graph_type == 'Bar Chart':
            graficar_barras(self.volatility_df, tickers_to_plot=selected_tickers)
        elif selected_graph_type == 'Heatmap':
            graficar_heatmap(self.volatility_df, tickers_to_plot=selected_tickers, metrics_to_plot=selected_metrics)
        elif selected_graph_type == 'Price with Volatility':
            if len(selected_tickers) > 1:
                 print("Please select only one ticker for 'Price with Volatility' plot.")
                 return
            graficar_precio_volatilidad(self.data, selected_tickers[0])

    def on_export_button_click(self, b):
        """Handles the export button click event."""
        exportar_resultados_csv(self.volatility_df)


# Instantiate the class and run the methods
tickers = ['AAPL', 'GOOG', 'MSFT', 'TSLA', 'BTC-USD', 'INVALID_TICKER'] # Example tickers including a crypto and an invalid one
radar = VolatilityRadar(tickers, start_date=date(2023, 6, 1), end_date=date.today()) # Example date range
radar.load_and_calculate()
radar.create_widgets()

/tmp/ipython-input-326481555.py:29: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-326481555.py:29: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-326481555.py:29: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-326481555.py:29: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-326481555.py:29: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Loading data...



/tmp/ipython-input-326481555.py:29: FutureWarning:

YF.download() has changed argument auto_adjust default to True

ERROR:yfinance:HTTP Error 404: 
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['INVALID_TICKER']: YFTzMissingError('possibly delisted; no timezone found')


Data loading complete.
Loaded data keys: dict_keys(['AAPL', 'GOOG', 'MSFT', 'TSLA', 'BTC-USD'])

Calculating volatility data...
Volatility calculation complete.

Calculated volatility data:


,daily_volatility,annualized_volatility,ATR,EWMA_volatility
AAPL,0.016920,0.268600,4.390000,0.015332
GOOG,0.018525,0.294074,6.473364,0.021482
MSFT,0.014160,0.224780,7.607856,0.010343
TSLA,0.038969,0.618621,15.666419,0.028629
BTC-USD,0.024574,0.390105,2124.272321,0.011614


Please select at least one ticker.

Generating Radar...



Generating Radar...


Volatility results exported to volatility_results.csv
Volatility results exported to volatility_results.csv
